# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

example with abc

In [12]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution
from edge_extractor import EdgeExtracter
from medusa_tools import *
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [13]:
bd.projects.delete_project("medusa_abc_example")
bd.projects.purge_deleted_directories()

2

In [14]:
bd.projects.set_current("medusa_abc_example")

bd.Database('medusa-bio').write({
            ('medusa-bio', "CO2"): {
                "type": "biosphere",
                "name": "carbon dioxide",
                "temporalis code": "co2",
            },
        },
)

bd.Database("background_2024").write(
    {
        ("background_2024", "C"): {
            'name': 'C',
            'location': 'somewhere',
            'reference product': 'C',
            "exchanges": [
                {
                    'amount': 1,
                    'type': 'production',
                    'input': ("background_2024", 'C'),
                },
                
                {
                    "amount": 5,
                    "input": ("medusa-bio", "CO2"),
                    "type": "biosphere",
                },
            ],       
        },
    },
    
)

bd.Database("background_2022").write(
    {
        ("background_2022", "C"): {
            'name': 'C',
            'location': 'somewhere',
            'reference product': 'C',
            "exchanges": [
                {
                    'amount': 1,
                    'type': 'production',
                    'input': ("background_2022", 'C'),
                },                   
                {
                    "amount": 15,
                    "input": ("medusa-bio", "CO2"),
                    "type": "biosphere",
                },
            ],   
        },
    },
    
)



bd.Database("foreground").write(
    {
        ("foreground", "A"): {
            'name': 'A',
            'location': 'somewhere',
            'reference product': 'A',
            "exchanges": [
                {
                    'amount': 1,
                    'type': 'production',
                    'input': ('foreground', 'A'),
                },                 
                {
                    "amount": 1,
                    "input": ("foreground", "B"),
                    'temporal_distribution':                     easy_timedelta_distribution(
                    start=-2,
                    end=0, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=2,
                ),
                    "type": "technosphere",
                },
            ],
        },
        
        ("foreground", "B"): {
            'name': 'B',
            'location': 'somewhere',
            'reference product': 'B',
            "exchanges": [
                {
                    'amount': 1,
                    'type': 'production',
                    'input': ('foreground', 'B'),
                },
                {
                    "amount": 1,
                    "input": ("background_2024", "C"),
                    "type": "technosphere",
                },
            ],
        },
    }
)

bd.Method(("GWP", "example")).write([
(("medusa-bio", "CO2"), 1),
])    

100%|██████████| 1/1 [00:00<00:00, 33554.43it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<00:00, 24385.49it/s]

Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<00:00, 34379.54it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 46603.38it/s]

Vacuuming database 


In [15]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [16]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 5.0


# `MEDUSA` LCA

A MEDUSA LCA builds upon a static LCA, but adds a temporal dimensions, linking to prospective LCA databases. Similarly to a `Temporalis LCA`, the supply chain graph is traversed, taking into account temporal distributions of the edges. 

For now, only the foreground system is assumed to have temporal distributions. Therefore, we define a filter function, that tells to EdgeExtracter (which is doing the actual graph traversal and saves the edges with respective timestamps), when a database that is known to have no temporal distributions (i.e., the prospective background databases) is reached, so that the traversal can be stopped.

In [17]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2023')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [18]:
eelca = EdgeExtracter(slca, edge_filter_function=filter_function)
timeline = eelca.build_edge_timeline()

Starting graph traversal
Calculation count: 2


Next, we define a dictionary containing the dates of our prospective background databases. Using this, we can create a timeline dataframe. 

The dates of the edges are mapped to the prospective background databases; interpolation is used for dates in between the dates of the background databases. The default is linear interpolation, another currently included option is "nearest", choosing the next best fitting database.

In [19]:
database_date_dict = {
            datetime.strptime("2020", "%Y"): 'background_2022',
            datetime.strptime("2024", "%Y"): 'background_2024',
        }

timeline_df = create_grouped_edge_dataframe(timeline, database_date_dict, interpolation_type="linear")
timeline_df

,year,producer,consumer,amount,date,interpolation_weights,producer_name,consumer_name,timestamp
0,2022,2,5,0.333333,2022-01-01,"{'background_2022': 0.4996577686516085, 'backg...",C,B,2022
1,2022,5,4,0.333333,2022-01-01,"{'background_2022': 0.4996577686516085, 'backg...",B,A,2022
2,2023,2,5,0.333333,2023-01-01,"{'background_2022': 0.2498288843258042, 'backg...",C,B,2023
3,2023,5,4,0.333333,2023-01-01,"{'background_2022': 0.2498288843258042, 'backg...",B,A,2023
4,2024,2,5,0.333333,2024-01-01,{'background_2024': 1},C,B,2024
5,2024,4,-1,1.0,2024-01-01,{'background_2024': 1},A,-1,2024
6,2024,5,4,0.333333,2024-01-01,{'background_2024': 1},B,A,2024


In [20]:
# check that the strings of the databases (values of database_date_dict) exist in the databases of the brightway2 project:
for db in database_date_dict.values():
    assert db in bd.databases, f"{db} not in your brightway2 project databases. Please check spelling of this database in database_date_dict and whether you are in the correct project."
else:
    print("All databases in database_date_dict exist as brightway project databases")

All databases in database_date_dict exist as brightway project databases


Now, we want to create a datapackage that takes care of relinking processes to our prospective databases. To do so, we need to provide the timeline dataframe, the dict of prospective databases and corresponding years, and a new dictionary that defines at which point in time our functional unit is assessed *(We can probably include this information in the database_date_dict in the future, but for now, this works)*.

In [21]:
demand_timing_dict = create_demand_timing_dict(timeline_df, demand)

dp = create_datapackage_from_edge_timeline(timeline_df, database_date_dict, demand_timing_dict)

{5: 2024}
4


KeyError: 4

Finally, we just have to reformat our input data for the LCA, add our datapackage containing the patches, and run the lca.

In [ ]:
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()

NameError: name 'prepare_medusa_lca_inputs' is not defined

Let's take a look at the results:

In [ ]:
print('New MEDUSA LCA Score:', lca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: 78053.71897810219
Old static LCA Score: 1.0


In [ ]:
lca.technosphere_matrix


<11x11 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [ ]:
lca.biosphere_matrix

<1x11 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [11]:
df= pd.DataFrame(lca.technosphere_matrix.toarray()) #for excel visualization
df.to_csv("test.csv", index=False)

second_items_list = [item[1] for item in lca.dicts.activity.reversed.items()]
second_items_list

NameError: name 'lca' is not defined

In [ ]:

for key in lca.activity_dict:
    print(bd.get_activity(key)['name'], bd.get_activity(key)["database"], ';') #BW does not find the "exploded nodes", because they exist only in the datapackages?
    

Electricity mix background_2023 ;
Electricity production, wind background_2023 ;
Electricity mix background_2020 ;
Electricity production, wind background_2020 ;
Heat production, hydrogen foreground ;
Hydrogen production, electrolysis foreground ;


UnknownObject: 